In [1]:
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
import io
import time
import numpy as np
import cv2
import os
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import NoSuchWindowException


In [2]:
def page_down(num):
    body = browser.find_element(By.CSS_SELECTOR, 'body')
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)
def page_down_detail():
    element = browser.find_elements(By.CSS_SELECTOR, 'body')
    element=element[1]
    element.send_keys(Keys.END)  
def search_iframe():
    browser.switch_to.default_content()
    browser.switch_to.frame("searchIframe")

def entry_iframe():
    browser.switch_to.default_content()
    WebDriverWait(browser, 5).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

In [3]:
region=['중구','연수구','동구','남동구','미추홀구','부평구','서구','계양구']
keward_text=[]
store=[]
review_txt=[]

In [4]:
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome()
for region_id in region:
    browser.get('https://map.naver.com/p?c=15.00,0,0,0,dh')
    time.sleep(2)
    incheon_input=browser.find_elements(By.CSS_SELECTOR,'input[type=text]')
    time.sleep(1)
    incheon_input[1].send_keys(f'인천광역시 {region_id} 카페')
    incheon_input[1].send_keys(Keys.ENTER)

    # 카페 이름 수집
    time.sleep(1)
    for i in range(1, 9):
        search_iframe()
        time.sleep(2)
        name_link = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
                                                                        f'#_pcmap_list_scroll_container > ul > li:nth-child({i}) > div.CHC5F > a.tzwk0 > div > div > span.place_bluelink.TYaxT')))
        browser.execute_script("arguments[0].click();", name_link)
        time.sleep(1)
        entry_iframe()

        # "리뷰 더보기" 버튼을 찾고 클릭
        element2 = browser.find_element(By.XPATH, '//span[text()="리뷰"]')
        browser.execute_script("arguments[0].click();", element2)
        time.sleep(1)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight/4);")
        for k in range(20):
            page_source = browser.page_source
            bs = BeautifulSoup(page_source, 'html.parser')
            time.sleep(1)
            start_time = time.time()  # 현재 시간 기록
            while True:
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
                try:
                    # 요소가 보일 때까지 기다림 (타임아웃은 10초로 설정)
                    element3 = WebDriverWait(browser, 10).until(
                        EC.visibility_of_element_located((By.XPATH, '//a[@class="fvwqf"]')))
                    # 요소가 보이면 반복문 종료
                    break
                except:
                    # 요소가 아직 보이지 않으면 계속 스크롤
                    break
                if time.time() - start_time > 5:
                    break
            try:
                browser.execute_script("arguments[0].click();", element3)
            except:
                break
        allDiv=bs.find_all('li',class_='owAeM')
        for review_box in allDiv:
            store.append(f'{region_id}')      
            try:
                review_text2 = review_box.find_all('div',class_='vg7Fp CyA_N')
                for j in review_text2:
                    review_bs=j.find('span',class_='zPfVt')
                    review_txt.append(review_bs.text) 
            except:
                review_txt.append('평가 없음')
            try:
                keward_box = review_box.find_all('div',class_='ERkm0')
                for k in keward_box:
                    try:
                        keward = k.find('span')
                        keward_text.append(keward.text)
                    except:
                        keward_text.append('평가 없음')
            except:
                keward_text.append('평가 없음')

In [9]:
cafe_df_reivew_keward = pd.DataFrame(data=[], columns=['id','review','keward'])
# k개의 리뷰를 수집합니다.
for i in range(len(review_txt)):
    tmp = []
    tmp.append(store[i])
    tmp.append(review_txt[i])
    tmp.append(keward_text[i])
    tmp = pd.DataFrame(data=[tmp], columns = cafe_df_reivew_keward.columns)
    cafe_df_reivew_keward = pd.concat([cafe_df_reivew_keward,tmp])
    

cafe_df_reivew_keward.reset_index(inplace=True, drop=True)
cafe_df_reivew_keward

,id,review,keward
0,중구,마시안제빵소 맛있어요 뷰맛집 사람들엄청 마나서 깜\n놀 연탄빵 유명하다 줄 서있...,빵이 맛있어요
1,중구,오션뷰 카페로 워낙 유명한 베이커리카페인가봐요 !\n겨우 좋은 자리 맡아서 티타임 ...,빵이 맛있어요
2,중구,연탄빵이 유명하다는 데 도저히 대기없인 구입이 어렵네요. 포기하고 다른 빵 먹었는데...,커피가 맛있어요
3,중구,평일 오후에는 손님도 많지않고 2층에서 조용히 바다\n를 볼수있어서 너무 좋았어요...,빵이 맛있어요
4,중구,드라이브겸 나들이 장소로 좋은\n영종도 마시안제빵소-!!,빵이 맛있어요
...,...,...,...
8102,서구,굿,야외 공간이 멋져요
8103,서구,이쁜 인테리어에 숨겨진 맛있는 음료와 디저트들 음\n식 하나하나에 정성이 느껴져 먹...,커피가 맛있어요
8104,서구,분위기도 좋고 맛도 좋고 친절하셔요,빵이 맛있어요
8105,서구,좋아요~,빵이 맛있어요


In [6]:
review_txt

['마시안제빵소 맛있어요 뷰맛집 사람들엄청 마나서 깜\n놀  연탄빵 유명하다  줄 서있는 사람 엄청 만나요 난 \n줄서있다 짤림 품절로 ㅋㅋ 넘 좋네요',
 '오션뷰 카페로 워낙 유명한 베이커리카페인가봐요 !\n겨우 좋은 자리 맡아서 티타임 즐겼어용 빵 가격도 생각보다 나쁘지않고 존맛이에요 :)',
 '연탄빵이 유명하다는 데 도저히 대기없인 구입이 어렵네요. 포기하고 다른 빵 먹었는데 맛이 괜찮았고, 다른 것 보단 오히려 갈매기 뷰맛집이었네요 ㅎㅎ',
 '평일 오후에는 손님도 많지않고 2층에서 조용히  바다\n를 볼수있어서 너무 좋았어요~ 바다가 보고싶으면 늘 \n마시안제빵소를 찾아 혼자 여유를 즐긴답니다~',
 '드라이브겸 나들이 장소로 좋은\n영종도 마시안제빵소-!!',
 '저녁에 가니 빵이 많이 품절되었어요 매장은 엄청 넓고 앞에 마시안해변이 보여서 뷰가 좋았어요',
 '빵도 맛있지만 뷰가 좋은 까페를 찾는다면 강추합니다 ',
 '연탄빵 사러 왔는데 매진되어 커피랑 빵만 먹고 있네\n요  두번왔는데  여전히 꽝 아쉽지만 걍 돌아갑니다',
 '빵 종류가 다양하고 주차 공간이 넓어서 드라이브 오\n기에 좋아요! 아이들을 데리고 오시는 가족 단위의 손\n님이 많습니다! 커피도 맛있고 디카페인도 가능...',
 '해변이랑 연결 되어 있어 나들이로 좋아요.',
 '뷰가 깔끔한 집입니다 아마 사람이 많아서 빵 하나 먹기에 힘들 때도 많아요.',
 '영종도 마시안 제비ㅏㅇ소를 이용 했어요',
 '바로 밑에, 해변으로 내려갈 수 있어서 아이랑 놀러가기 좋아요 ^^ 석양도 멋지고, 빵도 맛있고, 음료도 맛 있습니다 ♡ ',
 ' 자연 인테리어 오션 뻥뷰로 갈때마다 힐링 그 잡채!!! 베이커리와 음료도 맛있어요♡',
 '연탄빰 한번 쯤은 먹어볼만 하네요. 웨이팅은 기본입니다.',
 '근처에서 밥먹고 방문한 곳인데\n매장도 넓고 빵, 커피도 맛있습니다!\n일단 뷰가 너무 멋있어서 재방문 의사 있습니당\U0001fa75',
 '뷰 좋아요!!  그런데 연탄빵 대기 줄이 너무 혼

In [7]:
print(len(review_txt))
print(len(keward_text))

8107
10216


In [8]:
cafe_df['id'].unique()

array(['중구', '연수구', '동구', '남동구', '미추홀구', '부평구', '서구'], dtype=object)

In [12]:
cafe_df_reivew_keward.to_csv('cafe_df_review_keward.csv',index=False, encoding='utf-8-sig')

In [18]:
for i in region:
    print(cafe_df_reivew_keward['review'][cafe_df_reivew_keward['id']==f'{i}'].count())

1510
1443
1055
1499
1024
1296
280
0
